<a href="https://colab.research.google.com/github/nandrom/senalesIII/blob/main/Proyecto_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import numpy as np                     # Procesamiento numerico
import matplotlib.pyplot as plt        # graficos
import pandas as pd                    # Creacion y manejo de dataframes

from sklearn.cluster import KMeans     # para el clustering k-means
from sklearn.decomposition import PCA  # libreria para determinar las variables mas representativas

# Caracteristicas del dataset usado

In [8]:
num_datos = 683                        # cantidad de datos (muestras)
dent = int(num_datos*0.7)+1            # datos de entrenamiento   (verificar la cantidad de dtos para hacer la particion)
dpru = int(num_datos*0.3)              # datos de prueba
num_x = 9                              # numero de variables (caracteristicas)

kmax = 11                              # numero de clusters (se utilizara el algoritmo con diferente numero de clusters, esta variable determina el numero maximo de clusters, es necesario especificar el # maximo de clusters + 1)
itera = 300                            # numero maximo de iteraciones
tolerancia = 1e-6                      # tolerancia para la convergencia del algoritmo 

# Lectura del dataset

In [14]:
Datos = pd.read_excel('Datos.xlsx',dtype=int)         # leer el archivo de excel
df = pd.DataFrame(Datos)                              # Crear dataframe de los datos (matriz)
df

,D1,D2,D3,D4,D5,D6,D7,D8,D9,Cancer/NoCancer
0,5,1,1,1,2,1,3,1,1,1
1,5,4,4,5,7,10,3,2,1,1
2,3,1,1,1,2,2,3,1,1,1
3,6,8,8,1,3,4,3,7,1,1
4,4,1,1,3,2,1,3,1,1,1
...,...,...,...,...,...,...,...,...,...,...
678,3,1,1,1,2,1,2,1,1,1
679,1,1,1,1,2,1,1,1,1,1
680,4,1,4,1,2,1,1,1,1,1
681,1,1,1,1,2,1,1,1,1,1


# Se normaliza el dataset

In [15]:
#---------------------- normalizacion -----------------------------------------------------------------
def norm(df):                                         # Definicion de funcion de normalizacion
    return (df - df.min()) / ( df.max() - df.min())   # normalizacion

dfnorm = norm(df)                                     # llamado de la funcion normalizacion
dfnorm.to_excel("Datos normalizados.xlsx")            # se guardan los datos normalizados en un archivo de excel

#------ Visualizacion para verificar la normalizacion correcta ----------------------------------------

# ax = df['D6'].plot.hist(bins=20)
# ax.set_xlabel('Valores')
# ax.set_title('D1')
# plt.show()

# bx = dfnorm['D6'].plot.hist(bins=20)
# bx.set_xlabel('Valores')
# bx.set_title('D1')
# plt.show()

dfnorm

,D1,D2,D3,D4,D5,D6,D7,D8,D9,Cancer/NoCancer
0,0.444444,0.000000,0.000000,0.000000,0.111111,0.000000,0.222222,0.000000,0.0,1.0
1,0.444444,0.333333,0.333333,0.444444,0.666667,1.000000,0.222222,0.111111,0.0,1.0
2,0.222222,0.000000,0.000000,0.000000,0.111111,0.111111,0.222222,0.000000,0.0,1.0
3,0.555556,0.777778,0.777778,0.000000,0.222222,0.333333,0.222222,0.666667,0.0,1.0
4,0.333333,0.000000,0.000000,0.222222,0.111111,0.000000,0.222222,0.000000,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...
678,0.222222,0.000000,0.000000,0.000000,0.111111,0.000000,0.111111,0.000000,0.0,1.0
679,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.0,1.0
680,0.333333,0.000000,0.333333,0.000000,0.111111,0.000000,0.000000,0.000000,0.0,1.0
681,0.000000,0.000000,0.000000,0.000000,0.111111,0.000000,0.000000,0.000000,0.0,1.0


# Se asignan los datos de entrenamiento y los de prueba